# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from wordcloud import WordCloud, STOPWORDS
import plotly.graph_objects as go


In [ ]:
with open('../input/coleridgeinitiative-show-us-the-data/train/f8b03c87-9d1a-4f20-b76b-cb6c69d447b2.json') as f:
    sample = json.load(f)
    

In [ ]:
sample[:1]

for n in sample:
    print(n['section_title'])

In [ ]:
data_train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
data_train.sample(5)

In [ ]:
def get_text(filename, test=False):
    if test:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(filename))
    else:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(filename))
    text = " ".join(list(df['text']))
    return text

In [ ]:
data_train['text'] = data_train['Id'].apply(get_text)
data_train.sample(5)

In [ ]:

data_train['cleaned_label']=data_train['cleaned_label'].astype("category")
data_train.dtypes

In [ ]:


data_train['cleaned_label_cat']=data_train['cleaned_label'].cat.codes
data_train.head()

In [ ]:
test_files = os.listdir('../input/coleridgeinitiative-show-us-the-data/test')
test_files

In [ ]:
test = pd.DataFrame({'Id':test_files})
test['Id'] = test['Id'].apply(lambda x : x.split('.')[0])
test['text'] = test['Id'].apply(get_text, test=True)
test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data_train['text'])
X_train_counts.shape


In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, data_train['cleaned_label_cat'])

In [ ]:
X_new_counts = count_vect.transform(test['text'])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
X_new_tfidf.shape

In [ ]:
predicted = clf.predict(X_new_tfidf)

In [ ]:
print(predicted)

In [ ]:
labels= data_train[['cleaned_label' , 'cleaned_label_cat']]
print(labels[labels['cleaned_label_cat']==3])
print(labels[labels['cleaned_label_cat']==126])

In [ ]:
words_in_titles = list(data_train.pub_title.str.split(expand=True).stack())

wordcloud = WordCloud(stopwords = STOPWORDS,
                      background_color = "white",
                      width = 1000,
                      height = 500
                     ).generate(' '.join(words_in_titles))
plt.figure(1, figsize = (16, 12))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
text = ' '.join(data_train['text'].sample(frac=0.3))
wordcloud = WordCloud(background_color='white', stopwords=STOPWORDS, width=2560, height=1440).generate(text)

barplot_dim = (15, 15)
ax = plt.subplots(figsize=barplot_dim, facecolor='w')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# A text cleaning function
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()